In [29]:
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [26]:
dfe = pd.read_csv("../Data/Data_cleaned/emotion_cleaned_rudy.csv")[["target", "clean_text"]].dropna()

In [32]:
encoder = LabelEncoder()
dfe["cible"] = encoder.fit_transform(dfe["target"])

## Spliting des données

In [34]:
X_train, X_test, y_train, y_test = train_test_split(dfe["clean_text"], dfe["cible"], train_size=0.8, random_state=1, stratify=dfe["target"])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.5, stratify=y_train)

## Vectorisation

In [55]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [64]:
vectorizer =CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [65]:
X_train_t = vectorizer.transform(X_train)
X_test_t = vectorizer.transform(X_test)
X_val_t = vectorizer.transform(X_val)

## Neural Network

In [75]:
model = keras.Sequential()

initializer = keras.initializers.HeNormal()

model.add(keras.layers.Dense(20, input_dim=X_train_t.shape[1],kernel_initializer=initializer, activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.AlphaDropout(5))
model.add(keras.layers.Dense(6, activation="softmax"))

In [76]:
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(),
    optimizer= keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"]
)

In [77]:
history = model.fit(
    X_train_t.toarray(),
    y_train,
    epochs=100,
    callbacks=keras.callbacks.EarlyStopping(patience=3, monitor="val_loss"),
    batch_size=8,
    validation_data=(X_val_t.toarray(), y_val)
)

Epoch 1/100
1073/1073 [==============================] - 5s 4ms/step - loss: 1.7130 - accuracy: 0.3171 - val_loss: 1.4312 - val_accuracy: 0.4961
Epoch 2/100
1073/1073 [==============================] - 4s 4ms/step - loss: 1.0893 - accuracy: 0.6683 - val_loss: 0.9616 - val_accuracy: 0.7095
Epoch 3/100
1073/1073 [==============================] - 4s 4ms/step - loss: 0.6515 - accuracy: 0.8342 - val_loss: 0.6888 - val_accuracy: 0.7931
Epoch 4/100
1073/1073 [==============================] - 4s 4ms/step - loss: 0.3977 - accuracy: 0.9144 - val_loss: 0.5553 - val_accuracy: 0.8290
Epoch 5/100
1073/1073 [==============================] - 4s 4ms/step - loss: 0.2617 - accuracy: 0.9473 - val_loss: 0.4901 - val_accuracy: 0.8442
Epoch 6/100
1073/1073 [==============================] - 4s 4ms/step - loss: 0.1871 - accuracy: 0.9619 - val_loss: 0.4583 - val_accuracy: 0.8510
Epoch 7/100
1073/1073 [==============================] - 4s 4ms/step - loss: 0.1412 - accuracy: 0.9711 - val_loss: 0.4427 - val_ac

In [79]:
model.evaluate(X_test_t.toarray(), y_test)

135/135 [==============================] - 1s 3ms/step - loss: 0.4620 - accuracy: 0.8572


[0.46197882294654846, 0.8571761250495911]